In [ ]:
import numpy as np
import keras
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import pandas as pd
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
# import scikitplot
# from scikitplot.metrics import plot_confusion_matrix

In [ ]:
df_train = pd.read_csv('/work/archive (4) 2/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('/work/archive (4) 2/test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')

In [ ]:
df_train.Sentiment.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Sentiment, dtype: int64

In [ ]:
X=df_train['Input']

In [ ]:
lst=[]
for i in X:
  lst.append(len(i))

len1=pd.DataFrame(lst)
len1.describe()

,0
count,16000.000000
mean,96.845812
std,55.904953
min,7.000000
25%,53.000000
50%,86.000000
75%,129.000000
max,300.000000


In [ ]:
cts=[]
for i in range(7,301):
   ct=0
   for k in lst:
     if k==i:
       ct+=1
   cts.append(ct)

In [ ]:
tokenizer=Tokenizer(15212,lower=True,oov_token='UNK')
tokenizer.fit_on_texts(X)

In [ ]:
len(tokenizer.word_index)

15213

In [ ]:
X_train=tokenizer.texts_to_sequences(X)
X_train_pad=pad_sequences(X_train,maxlen=80,padding='post')

In [ ]:
df_train['Sentiment']=df_train.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
Y_train=df_train['Sentiment'].values

In [ ]:
Y_train_f=to_categorical(Y_train)

In [ ]:
Y_train_f[:6]

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
df_val=pd.read_csv('/work/archive (4) 2/val.txt',header=None,sep=';',names=['Input','Sentiment'],encoding='utf-8')

In [ ]:
X_val=df_val['Input']
Y_val=df_val.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
X_val_f=tokenizer.texts_to_sequences(X_val)
X_val_pad=pad_sequences(X_val_f,maxlen=80,padding='post')

In [ ]:
Y_val_f=to_categorical(Y_val)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional,Dense,Embedding,Dropout

In [ ]:
model=Sequential()
model.add(Embedding(15212,64,input_length=80))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(80,return_sequences=True)))
model.add(Bidirectional(LSTM(160)))
model.add(Dense(6,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 64)            973568    
_________________________________________________________________
dropout (Dropout)            (None, 80, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 80, 160)           92800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 320)               410880    
_________________________________________________________________
dense (Dense)                (None, 6)                 1926      
Total params: 1,479,174
Trainable params: 1,479,174
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit(X_train_pad,Y_train_f,epochs=12,validation_data=(X_val_pad,Y_val_f))

Epoch 1/12
500/500 [==============================] - 220s 431ms/step - loss: 1.4713 - accuracy: 0.4023 - val_loss: 0.6834 - val_accuracy: 0.7525
Epoch 2/12
500/500 [==============================] - 217s 435ms/step - loss: 0.5257 - accuracy: 0.8148 - val_loss: 0.2567 - val_accuracy: 0.9175
Epoch 3/12
500/500 [==============================] - 215s 429ms/step - loss: 0.2641 - accuracy: 0.9069 - val_loss: 0.2235 - val_accuracy: 0.9210
Epoch 4/12
500/500 [==============================] - 216s 432ms/step - loss: 0.1985 - accuracy: 0.9292 - val_loss: 0.2077 - val_accuracy: 0.9190
Epoch 5/12
500/500 [==============================] - 214s 428ms/step - loss: 0.1486 - accuracy: 0.9445 - val_loss: 0.1701 - val_accuracy: 0.9240
Epoch 6/12
500/500 [==============================] - 214s 429ms/step - loss: 0.1336 - accuracy: 0.9493 - val_loss: 0.1805 - val_accuracy: 0.9250
Epoch 7/12
500/500 [==============================] - 215s 429ms/step - loss: 0.1089 - accuracy: 0.9594 - val_loss: 0.1511 -

In [ ]:
X_test=df_test['Input']
Y_test=df_test.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})

In [ ]:
X_test_f=tokenizer.texts_to_sequences(X_test)
X_test_pad=pad_sequences(X_test_f,maxlen=80,padding='post')

In [ ]:
Y_test_f=to_categorical(Y_test)
X_test_pad.shape
Y_test_f[:7]

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
model.evaluate(X_test_pad,Y_test_f)

63/63 [==============================] - 7s 111ms/step - loss: 0.1807 - accuracy: 0.9295


[0.1806565672159195, 0.9294999837875366]

In [ ]:
Y_pred=model.predict_classes(X_test_pad)

/shared-libs/python3.7/py/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       695
           1       0.92      0.95      0.94       275
           2       0.89      0.75      0.82       159
           3       0.98      0.96      0.97       581
           4       0.94      0.84      0.89       224
           5       0.67      0.88      0.76        66

    accuracy                           0.93      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.93      0.93      0.93      2000



In [ ]:
def get_key(value):
    dictionary={'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5}
    for key,val in dictionary.items():
          if (val==value):
            return key

In [ ]:
def predict(sentence):
  sentence_lst=[]
  sentence_lst.append(sentence)
  sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
  sentence_padded=pad_sequences(sentence_seq,maxlen=80,padding='post')
  ans=get_key(model.predict_classes(sentence_padded))
  print("The emotion predicted is",ans)

In [ ]:
predict(str(input('Enter a sentence : ')))

KeyboardInterrupt: Interrupted by user

In [ ]:
model.save('lstm_sentiment.h5')

In [ ]:
li=["I want to punch"]
sentence_seq=tokenizer.texts_to_sequences(li)
sentence_padded=pad_sequences(sentence_seq,maxlen=80,padding='post')
ans=get_key(model.predict_classes(sentence_padded))
print(ans)

anger


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=44141780-bb75-42f9-884a-3d1e7c3f3442' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>